In [ ]:
%pip install pyspark==3.5.1 --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from pyspark.sql import SparkSession
import subprocess

hdfs_path = "C:/hadoop/bin/hdfs.cmd"  # chemin exact de hdfs.cmd sur ton PC

# Supprimer le dossier si existant
subprocess.run([hdfs_path, "dfs", "-rm", "-r", "/output/test.txt"])


# Initialiser SparkSession avec support Hadoop
spark = SparkSession.builder \
    .appName("PySpark HDFS Example") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .config("spark.hadoop.fs.hdfs.impl", "org.apache.hadoop.hdfs.DistributedFileSystem") \
    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .config("spark.hadoop.fs.hdfs.impl.disable.cache", "true") \
    .getOrCreate()

sc = spark.sparkContext

sc

<SparkContext master=local[*] appName=PySpark HDFS Example>

In [15]:
# --- Écriture dans HDFS ---

# Chemin HDFS où écrire
hdfs_write_path = "hdfs://localhost:9000/test/test.txt"

In [16]:
# Exemple de données
data = ["Bonjour HDFS depuis PySpark!", "Test d'écriture de fichier"]

# Créer un RDD
rdd = sc.parallelize(data)

In [17]:
rdd.collect()

['Bonjour HDFS depuis PySpark!', "Test d'écriture de fichier"]

In [18]:
# Écrire le RDD comme fichier texte dans HDFS

rdd.saveAsTextFile("hdfs://localhost:9000/output/test.txt")
print(f"Fichier écrit dans HDFS à l'emplacement : hdfs://localhost:9000/output/test.txt")

Fichier écrit dans HDFS à l'emplacement : hdfs://localhost:9000/output/test.txt


In [19]:
# --- Lecture depuis HDFS ---

# Lire le fichier texte de HDFS dans un RDD
rdd_read = sc.textFile("hdfs://localhost:9000/output/test.txt")

# Collecter et afficher le contenu
content = rdd_read.collect()
print("Contenu lu depuis HDFS:")
for line in content:
    print(line)

Contenu lu depuis HDFS:
Bonjour HDFS depuis PySpark!
Test d'écriture de fichier


In [20]:
# Fermer la session Spark
sc.stop()